In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pandas as pd
import numpy as np
import re
import os
from collections import defaultdict
from sortedcontainers import SortedDict
from modules.acronym import Acronym

In [17]:
qs = pd.read_excel('./output/Alloys_Unmapped.xlsx')
qs['Alloy'] = qs['Alloy'].astype('string')
qs

,Alloy,Processing condition,Testing condition,Property,Value,Unit,doi,original property,caption,footer,table_prop,Processed Alloy Name,Type,Conventional,Category
0,Cu-Ge,-,Temperature(in K): 1173,Mixing Enthalpy,(3) -7.6-41.9*x Cu +620*x Cu 2-4330*x Cu 3+1...,kJ/mol,./10.1016.S0021-9614(03)00157-5,Thermodynamic function,Evaluated parameters for the polynomial repres...,{'a': 'The standard component state is liquid ...,"Thermodynamic function, Dmix H (Cu-Ge), Dmix G...",Cu-Ge,alloy,unconventional,Unconventional
1,Al-Cu-Ge,-,T=1380 K; x Al: 0.4; x Ge:x Cu: 0.7:0.3,Mixing Enthalpy,-6.2±0.2,kJ/mol,./10.1016.S0021-9614(03)00157-5,D mix H Al,The partial for aluminium and integral enthalp...,NaN,"D mix H Al, Dmix H",Al-Cu-Ge,alloy,unconventional,Unconventional
2,Al-Cu-Ge,-,T=1380 K; x Al: 0.4; x Ge:x Cu: 0.9:0.1,Mixing Enthalpy,-6.6±0.3,kJ/mol,./10.1016.S0021-9614(03)00157-5,D mix H Al,The partial for aluminium and integral enthalp...,NaN,"D mix H Al, Dmix H",Al-Cu-Ge,alloy,unconventional,Unconventional
3,Al-Cu-Ge,-,T=1380 K; x Al: 0.5; x Ge:x Cu: 0.3:0.7,Mixing Enthalpy,-3.0±0.3,kJ/mol,./10.1016.S0021-9614(03)00157-5,D mix H Al,The partial for aluminium and integral enthalp...,NaN,"D mix H Al, Dmix H",Al-Cu-Ge,alloy,unconventional,Unconventional
4,Al-Cu-Ge,-,T=1380 K; x Al: 0.5; x Ge:x Cu: 0.5:0.5,Mixing Enthalpy,-3.4±0.3,kJ/mol,./10.1016.S0021-9614(03)00157-5,D mix H Al,The partial for aluminium and integral enthalp...,NaN,"D mix H Al, Dmix H",Al-Cu-Ge,alloy,unconventional,Unconventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148063,Aged,-,Temperature(in K): 320,Coercivity,264+-10,Oe,./10.1016.j.ultramic.2010.12.001,Coercivity at 320K (Oe),"The saturation magnetization, remanance ratio ...",NaN,Aged,Aged,alloy,unconventional,Unconventional
148064,Aged,-,Temperature(in K): 320,Remanence,8.41,%,./10.1016.j.ultramic.2010.12.001,Remanance ratio at 320K (%),"The saturation magnetization, remanance ratio ...",NaN,Aged,Aged,alloy,unconventional,Unconventional
148065,AlCoCrFeNi,-,Region: B,Chemical Composition,Al: 4.9±3.6; Co: 22.3±0.6; Cr: 35.7±5.1; Fe: 2...,at%,./10.1016.j.ultramic.2012.12.015,Fe [at%],Microchemical composition of the investigated ...,NaN,AlCoCrFeNi,AlCoCrFeNi,alloy,conventional,Conventional
148066,AlCoCrFeNi,-,Region: A,Chemical Composition,Al: 32.0±3.4; Co: 22.6±2.0; Cr: 4.9±1.4; Fe: 1...,at%,./10.1016.j.ultramic.2012.12.015,Fe [at%],Microchemical composition of the investigated ...,NaN,AlCoCrFeNi,AlCoCrFeNi,alloy,conventional,Conventional


In [18]:
final_output = pd.DataFrame()


for group in qs.groupby(by='doi'):
    doi = group[0]
    df = group[1].copy()

    composition = df[(df['Property'].str.contains('Composition', na=False))]

    mapping = defaultdict(set)

    for _, row in composition.iterrows():
        alloy = row['Alloy'] if not(pd.isna(row['Alloy'])) else "NaN"
        val = row['Value'] if not(pd.isna(row['Value'])) else "NaN"
        unit = row['Unit'] if not(pd.isna(row['Unit'])) else "NaN"
        processing = row['Processing condition'] if not(pd.isna(row['Processing condition'])) else "NaN"
        testing = row['Testing condition'] if not(pd.isna(row['Testing condition'])) else "NaN"
        nominal = "Nominal" if ("nominal" in processing.lower() or "nominal" in testing.lower()) else "NaN"
        mapping[alloy].add((val, unit, nominal))

    one_one_mapping = dict()
    for k, v in mapping.items():
        if len(v) == 1:
            val, unit, _ = next(iter(v))
            one_one_mapping[k] = Acronym(k, val, unit)
        else:
            for val, unit, nominal in v:
                if nominal == "Nominal":
                    one_one_mapping[k] = Acronym(k, val, unit)
                    break

    def corrected_alloy_comp(row):
        key = row['Alloy']
        if key in one_one_mapping:
            return one_one_mapping[key].serialized_composition
        else:
            return "105_InvalidMappingError"
    
    def get_unit(row):
        atomic_percent_terms = [
            'mol%',
            'mole fraction %',
            'atomic mole fraction',
            'at.%',
            'atomic%',
            'mole fraction',
            'mol',
            '% at.',
            'molar ratio',
            'atomic fraction',
            '% at',
            'Atomic%',
            'mol.%',
            'at%'
        ]

        weight_percent_terms = [
            'wt%',
            'w%', 
            '% wt', 
            '% wt.', 
            ' wt%', 
            'mass%'
        ]

        key = row['Alloy']
        mapped_alloy = row['Alloy_new']
        if key in one_one_mapping and (not re.match(r'^\d{3}', mapped_alloy)):
            unit = one_one_mapping[key].unit
            if unit in atomic_percent_terms:
                return 'at%'
            elif unit in weight_percent_terms:
                return 'wt%'
            else:
                return unit
        else:
            return "201_UnitNotFound"

    df['Alloy_new'] = df.apply(corrected_alloy_comp, axis=1)
    df['Composition_unit'] = df.apply(get_unit, axis=1)

    final_output = pd.concat([final_output, df]) if not final_output.empty else df

final_output.sort_values(by='doi', inplace=True)

In [19]:
results = final_output[~final_output['Alloy_new'].str.match(r"^\d{3}")]
results

,Alloy,Processing condition,Testing condition,Property,Value,Unit,doi,original property,caption,footer,table_prop,Processed Alloy Name,Type,Conventional,Category,Alloy_new,Composition_unit
720,V0.6Ge0.4,Before KEMS experiment; annealed at 1000degC,-,Chemical Composition,V: 64.6; Ge: 35.4,at%,./10.1016.S0925-8388(99)00801-4,Composition of the phases,Vanadium germanide samples characteristics by ...,{'a': 'Solid solution of Ge in V.'},Ge0.4V0.6,Ge0.4V0.6,alloy,conventional,Conventional,Ge35.4V64.6,at%
718,V0.9Ge0.1,After KEMS experiment,-,Chemical Composition,V: 93.5; Ge: 6.5,at%,./10.1016.S0925-8388(99)00801-4,Composition of the phases,Vanadium germanide samples characteristics by ...,{'a': 'Solid solution of Ge in V.'},Ge0.1V0.9,Ge0.1V0.9,alloy,conventional,Conventional,Ge6.5V93.5,at%
1315,Al02Ti10,-,Matrix,Chemical Composition,Al: 0.7; Co: 23.6; Cr: 26.2; Fe: 22.1; Ni: 12....,at%,./10.1016.j.actamat.2011.06.041,Nominal,Nominal and EDS-determined compositions (in at...,NaN,Al02Ti10,Al02Ti10,alloy,conventional,Conventional,Al3.2Co24.2Cr16.1Fe16.1Ni24.2Ti16.1,at%
1317,Al00Ti05,-,Matrix,Chemical Composition,Co: 27.0; Cr: 19.3; Fe: 18.8; Ni: 26.2; Ti: 8.6,at%,./10.1016.j.actamat.2011.06.041,Co,Nominal and EDS-determined compositions (in at...,NaN,Al00Ti05,Al00Ti05,alloy,conventional,Conventional,Co27.3Cr18.2Fe18.2Ni27.3Ti9.1,at%
1318,Al00Ti05,-,e,Chemical Composition,Co: 23.5; Cr: 5.6; Fe: 7.6; Ni: 43.3; Ti: 20.1,at%,./10.1016.j.actamat.2011.06.041,Al,Nominal and EDS-determined compositions (in at...,NaN,Al00Ti05,Al00Ti05,alloy,conventional,Conventional,Co27.3Cr18.2Fe18.2Ni27.3Ti9.1,at%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147956,AlCuNiTiZr,-,non-annealed;post-corrosion,Chemical Composition,Al: 29; Cu: 20; Ni: 17; Ti: 17; Zr: 17,at%,./10.1016.j.tsf.2021.138707,Table Data: element(at%),composition of AlCuNiTiZr films.,NaN,AlCuNiTiZr,AlCuNiTiZr,alloy,conventional,Conventional,Al29.0Cu20.0Ni17.0Ti17.0Zr17.0,at%
147958,AlCuNiTiZr,-,non-annealed;pre-corrosion,Chemical Composition,Al: 29; Cu: 20; Ni: 17; Ti: 17; Zr: 17,at%,./10.1016.j.tsf.2021.138707,Table Data: element(at%),composition of AlCuNiTiZr films.,NaN,AlCuNiTiZr,AlCuNiTiZr,alloy,conventional,Conventional,Al29.0Cu20.0Ni17.0Ti17.0Zr17.0,at%
147957,AlCuNiTiZr,-,500 annealed;post-corrosion,Chemical Composition,Al: 29; Cu: 20; Ni: 17; Ti: 17; Zr: 17,at%,./10.1016.j.tsf.2021.138707,Table Data: element(at%),composition of AlCuNiTiZr films.,NaN,AlCuNiTiZr,AlCuNiTiZr,alloy,conventional,Conventional,Al29.0Cu20.0Ni17.0Ti17.0Zr17.0,at%
147955,AlCuNiTiZr,-,500 annealed;pre-corrosion,Chemical Composition,Al: 29; Cu: 20; Ni: 17; Ti: 17; Zr: 17,at%,./10.1016.j.tsf.2021.138707,Table Data: element(at%),composition of AlCuNiTiZr films.,NaN,AlCuNiTiZr,AlCuNiTiZr,alloy,conventional,Conventional,Al29.0Cu20.0Ni17.0Ti17.0Zr17.0,at%


In [20]:
results['Category'].value_counts()

Category
Conventional      5976
Unconventional    5892
Name: count, dtype: int64

In [21]:
final_output.to_excel('./output/Alloys_Mapped_New.xlsx', index=False)